4. Modular AI-Augmented LCA Model ♻️
Scenario: We're building an AI-enhanced Life Cycle Assessment (LCA) model.

Tasks:

Refactor to Modules: The Python class below is monolithic. Refactor it into separate classes (DataIngestor, EmissionFactorMapper, SupplyChainModel, ImpactCalculator, UncertaintyAnalyzer).
Implement Uncertainty: Add a basic Monte Carlo simulation method to the UncertaintyAnalyzer class to estimate the range of potential LCA results, given uncertainty ranges for emission factors.
Design: Explain how AI (e.g., NLP for factor extraction, GNNs for supply chains) could enhance each module.
Code Sample (Python Class - Needs Refactoring & Monte Carlo!):

In [8]:
import random

class MonolithicLCA:
    def __init__(self, product_data):
        self.data = product_data
        # PROBLEM: All logic is crammed here - ingestion, mapping, calculation...
        self.factors = {'steel': 2.1, 'transport': 0.15}

    def run_lca(self):
        steel_emissions = self.data['steel_kg'] * self.factors['steel']
        transport_emissions = self.data['transport_km'] * self.factors['transport']
        total = steel_emissions + transport_emissions
        # PROBLEM: No modularity, no uncertainty handling.
        print(f"Total Emissions: {total} tCO2e")
        return total

# data = {'steel_kg': 1000, 'transport_km': 500}
# lca = MonolithicLCA(data); lca.run_lca()
# Our Take: A complex LCA needs modularity. Refactor this and add uncertainty analysis.

In [9]:
import pandas as pd

class DataIngestor:
    """
    Ingests and validates product data for LCA analysis.
    """
    def __init__(self, product_data: dict):
        """
        Initialize with product data dictionary.
        
        Parameters:
        product_data (dict): Dictionary with keys like 'steel_kg', 'transport_km'.
        """
        self.product_data = product_data
        self._validate_data()
    
    def _validate_data(self):
        """Validate input data for required keys and types."""
        required_keys = ['steel_kg', 'transport_km']
        for key in required_keys:
            if key not in self.product_data:
                raise ValueError(f"Missing required key: {key}")
            if not isinstance(self.product_data[key], (int, float)) or self.product_data[key] < 0:
                raise ValueError(f"Invalid value for {key}: must be a non-negative number")
    
    def get_data(self):
        """Return validated product data."""
        return self.product_data

In [10]:
class EmissionFactorMapper:
    """
    Manages emission factors for materials and activities.
    """
    def __init__(self, factors=None):
        """
        Initialize with emission factors dictionary.
        
        Parameters:
        factors (dict): Dictionary with keys like 'steel', 'transport' and tCO2e values.
                        Defaults to {'steel': 2.1, 'transport': 0.15}.
        """
        self.factors = factors if factors is not None else {'steel': 2.1, 'transport': 0.15}
        self._validate_factors()
    
    def _validate_factors(self):
        """Validate emission factors for required keys and positive values."""
        required_keys = ['steel', 'transport']
        for key in required_keys:
            if key not in self.factors:
                raise ValueError(f"Missing emission factor for: {key}")
            if not isinstance(self.factors[key], (int, float)) or self.factors[key] <= 0:
                raise ValueError(f"Invalid emission factor for {key}: must be positive")
    
    def get_factor(self, key):
        """Return emission factor for a given key."""
        return self.factors.get(key, 0)
    
    def get_factors_with_uncertainty(self, uncertainty_percent=10):
        """
        Return emission factors with uncertainty ranges.
        
        Parameters:
        uncertainty_percent (float): Percentage uncertainty (e.g., 10 for ±10%).
        
        Returns:
        dict: Dictionary with keys and tuples (mean, lower_bound, upper_bound).
        """
        uncertainty = uncertainty_percent / 100
        return {
            key: (value, value * (1 - uncertainty), value * (1 + uncertainty))
            for key, value in self.factors.items()
        }

In [11]:
class SupplyChainModel:
    """
    Models supply chain processes and maps them to emission factors.
    """
    def __init__(self, data_ingestor, emission_factor_mapper):
        """
        Initialize with data ingestor and emission factor mapper.
        
        Parameters:
        data_ingestor (DataIngestor): Provides product data.
        emission_factor_mapper (EmissionFactorMapper): Provides emission factors.
        """
        self.data = data_ingestor.get_data()
        self.factor_mapper = emission_factor_mapper
    
    def calculate_process_emissions(self):
        """
        Calculate emissions for each process (e.g., steel, transport).
        
        Returns:
        dict: Dictionary with process names and their emissions (tCO2e).
        """
        return {
            'steel_emissions': self.data['steel_kg'] * self.factor_mapper.get_factor('steel'),
            'transport_emissions': self.data['transport_km'] * self.factor_mapper.get_factor('transport')
        }

In [12]:
class ImpactCalculator:
    """
    Calculates total environmental impact (emissions) from supply chain processes.
    """
    @staticmethod
    def calculate_total_emissions(process_emissions):
        """
        Calculate total emissions from process emissions.
        
        Parameters:
        process_emissions (dict): Dictionary with process emissions (tCO2e).
        
        Returns:
        float: Total emissions in tCO2e.
        """
        return sum(process_emissions.values())
    
    @staticmethod
    def report_emissions(total_emissions, process_emissions):
        """
        Print a report of total and process-specific emissions.
        
        Parameters:
        total_emissions (float): Total emissions in tCO2e.
        process_emissions (dict): Dictionary with process emissions (tCO2e).
        """
        print(f"Total Emissions: {total_emissions:.2f} tCO2e")
        for process, emissions in process_emissions.items():
            print(f"{process}: {emissions:.2f} tCO2e")

In [13]:
import numpy as np

class UncertaintyAnalyzer:
    """
    Analyzes uncertainty in LCA results using Monte Carlo simulation.
    """
    def __init__(self, supply_chain_model, emission_factor_mapper):
        """
        Initialize with supply chain model and emission factor mapper.
        
        Parameters:
        supply_chain_model (SupplyChainModel): Provides process emissions.
        emission_factor_mapper (EmissionFactorMapper): Provides emission factors with uncertainty.
        """
        self.supply_chain_model = supply_chain_model
        self.factor_mapper = emission_factor_mapper
        self.data = self.supply_chain_model.data
    
    def run_monte_carlo(self, n_simulations=1000, uncertainty_percent=10):
        """
        Run Monte Carlo simulation to estimate LCA result uncertainty.
        
        Parameters:
        n_simulations (int): Number of simulations to run.
        uncertainty_percent (float): Percentage uncertainty for emission factors.
        
        Returns:
        dict: Statistics including mean, 5th percentile, and 95th percentile of total emissions.
        """
        factors_with_uncertainty = self.factor_mapper.get_factors_with_uncertainty(uncertainty_percent)
        total_emissions = []
        
        for _ in range(n_simulations):
            # Sample emission factors within uncertainty ranges
            sampled_factors = {
                key: np.random.uniform(low= bounds[1], high=bounds[2])
                for key, bounds in factors_with_uncertainty.items()
            }
            
            # Temporarily update emission factors
            original_factors = self.factor_mapper.factors.copy()
            self.factor_mapper.factors.update(sampled_factors)
            
            # Calculate emissions with sampled factors
            process_emissions = self.supply_chain_model.calculate_process_emissions()
            total = ImpactCalculator.calculate_total_emissions(process_emissions)
            total_emissions.append(total)
            
            # Restore original factors
            self.factor_mapper.factors = original_factors
        
        # Compute statistics
        return {
            'mean': np.mean(total_emissions),
            'p5': np.percentile(total_emissions, 5),
            'p95': np.percentile(total_emissions, 95)
        }

In [14]:
# Sample product data
product_data = {'steel_kg': 1000, 'transport_km': 500}

# Initialize module
data_ingestor = DataIngestor(product_data)
emission_factor_mapper = EmissionFactorMapper()
supply_chain_model = SupplyChainModel(data_ingestor, emission_factor_mapper)
impact_calculator = ImpactCalculator()
uncertainty_analyzer = UncertaintyAnalyzer(supply_chain_model, emission_factor_mapper)

# Run deterministic LCA
process_emissions = supply_chain_model.calculate_process_emissions()
total_emissions = impact_calculator.calculate_total_emissions(process_emissions)
impact_calculator.report_emissions(total_emissions, process_emissions)

# Run Monte Carlo simulation
uncertainty_results = uncertainty_analyzer.run_monte_carlo(n_simulations=1000, uncertainty_percent=10)
print("\nMonte Carlo Uncertainty Results:")
print(f"Mean Emissions: {uncertainty_results['mean']:.2f} tCO2e")
print(f"5th Percentile: {uncertainty_results['p5']:.2f} tCO2e")
print(f"95th Percentile: {uncertainty_results['p95']:.2f} tCO2e")

Total Emissions: 2175.00 tCO2e
steel_emissions: 2100.00 tCO2e
transport_emissions: 75.00 tCO2e

Monte Carlo Uncertainty Results:
Mean Emissions: 2181.83 tCO2e
5th Percentile: 1987.30 tCO2e
95th Percentile: 2363.50 tCO2e


## 3. AI‑Enhanced Modules

### DataIngestor (with NLP)
- **Enhancement:** BERT‑based NER extracts quantities/units (“1000 kg steel”, “500 km transport”) from unstructured reports.  
- **Benefit:** Automates data entry and reduces manual errors.  
- **Example:** Parses “Product uses 1000 kg steel” → `steel_kg = 1000`.  

### EmissionFactorMapper (with NLP + Knowledge Graph)
- **Enhancement:** LLM scrapes factors (“steel: 2.1 tCO₂e/kg”) from literature and stores them in a graph database (e.g., Neo4j).  
- **Benefit:** Dynamic updates, regional specificity, semantic querying.  
- **Example:** Query “steel factor in Europe” → `2.1 tCO₂e/kg`.  

### SupplyChainModel (with GNN)
- **Enhancement:** Models multi‑tier supply chains as graphs; a GNN propagates and predicts missing emissions across nodes.  
- **Benefit:** Handles complex, nested supplier networks and captures indirect emissions.  
- **Example:** Predicts steel‑production emissions when only transport data is available.  

### ImpactCalculator (with ML Regression)
- **Enhancement:** RandomForestRegressor trained on historical LCA data fills gaps and calibrates deterministic calculations.  
- **Benefit:** Improves accuracy with partial inputs and provides confidence scores.  
- **Example:** Estimates total emissions given only `steel_kg`.  

### UncertaintyAnalyzer (with Bayesian Neural Networks (BNNs) / Gaussian Process)
- **Enhancement:** Bayesian NN or GP models uncertainty in emission factors, replacing brute‑force Monte Carlo with efficient probabilistic sampling.  
- **Benefit:** Fewer simulations, adaptive uncertainty quantification, richer probabilistic outputs.  
- **Example:** Outputs total emissions distribution (mean ± std) with 90% credible interval.  
